[View in Colaboratory](https://colab.research.google.com/github/rdenadai/TxtP-Study-Notebooks/blob/master/notebooks/text_classification_example.ipynb)

## Análise e Validação de Textos em Português


### Referências:

 - [NLTK](http://www.nltk.org/howto/portuguese_en.html)
 - [spaCy](https://spacy.io/usage/spacy-101)
 - [Utilizando processamento de linguagem natural para criar uma sumarização automática de textos](https://medium.com/@viniljf/utilizando-processamento-de-linguagem-natural-para-criar-um-sumariza%C3%A7%C3%A3o-autom%C3%A1tica-de-textos-775cb428c84e)
 - [Latent Semantic Analysis (LSA) for Text Classification Tutorial](http://mccormickml.com/2016/03/25/lsa-for-text-classification-tutorial/)
 - [Topic Modeling with LSA, PLSA, LDA & lda2Vec](https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05)
 - [Unsupervised Emotion Detection from Text using Semantic and Syntactic Relations](http://www.cse.yorku.ca/~aan/research/paper/Emo_WI10.pdf)

### Instalação

In [1]:
!pip install -U spacy
!python -m spacy download en
!python -m spacy download pt
# !pip install feedparser

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.0.12)
    100% |████████████████████████████████| 37.4MB 39.2MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

    100% |████████████████████████████████| 38.7MB 23.1MB/s 
  Running setup.py install for pt-core-news-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')



In [2]:
# Download Oplexicon
!rm -rf wget-log*
!rm -rf oplexicon_v3.0
!wget -O oplexicon_v3.0.zip https://github.com/rdenadai/sentiment-analysis-2018-president-election/blob/master/dataset/oplexicon_v3.0.zip?raw=true
!unzip oplexicon_v3.0.zip
!ls -lh


Redirecting output to ‘wget-log’.
Archive:  oplexicon_v3.0.zip
  inflating: oplexicon_v3.0/lexico_v3.0.txt  
  inflating: oplexicon_v3.0/README   
total 120K
drwxr-xr-x 2 root root 4.0K Oct  6 00:48 oplexicon_v3.0
-rw-r--r-- 1 root root 102K Oct  6 00:48 oplexicon_v3.0.zip
drwxr-xr-x 2 root root 4.0K Sep 28 23:32 sample_data
-rw-r--r-- 1 root root 1.6K Oct  6 00:48 wget-log


### Imports

In [30]:
import nltk

nltk.download('rslp')
nltk.download('averaged_perceptron_tagger')
nltk.download('floresta')
nltk.download('mac_morpho')
nltk.download('machado')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

import concurrent.futures
import codecs
import re
import pprint
from random import shuffle
from string import punctuation
import copy
from unicodedata import normalize

import numpy as np
from scipy.sparse.linalg import svds
from scipy.linalg import svd
import pandas as pd
import spacy
from spacy.lang.pt.lemmatizer import LOOKUP

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.utils.extmath import randomized_svd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import floresta as flt
from nltk.corpus import machado as mch
from nltk.corpus import mac_morpho as mcm


nlp = spacy.load('pt')
pp = pprint.PrettyPrinter(indent=4)
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Package floresta is already up-to-date!
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk

### Functions

In [0]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')


def load_oplexicon_data(filename):
    spacy_conv = {
        'adj': 'ADJ',
        'n': 'NOUN',
        'vb': 'VERB',
        'det': 'DET',
        'emot': 'EMOT',
        'htag': 'HTAG'
    }
    
    data = {}
    with codecs.open(filename, 'r', 'UTF-8') as hf:
        lines = hf.readlines()
        for line in lines:
            info = line.lower().split(',')
            if len(info[0].split()) <= 1:
                info[1] = [spacy_conv.get(tag) for tag in info[1].split()]
                word, tags, sent = info[:3]
                if 'HTAG' not in tags and 'EMOT' not in tags:
                    word = remover_acentos(word.replace('-se', ''))
                    word = LOOKUP.get(word, word)
                    # stem = stemmer.stem(word)
                    if word in data:
                        data[word] += [{
                            'word': [word],
                            'tags': tags,
                            'sentiment': sent
                        }]
                    else:
                        data[word] = [{
                            'word': [word],
                            'tags': tags,
                            'sentiment': sent
                        }]
    return data

### Usage

In [35]:
frase = u"Gostaria de saber mais informações sobre a Amazon. Uma excelente loja de produtos online!".lower()
doc = nlp(remover_acentos(frase))
pp.pprint([(w.text, w.pos_) for w in doc])

[   ('gostaria', 'VERB'),
    ('de', 'ADP'),
    ('saber', 'VERB'),
    ('mais', 'ADV'),
    ('informacoes', 'NOUN'),
    ('sobre', 'ADP'),
    ('a', 'DET'),
    ('amazon', 'NOUN'),
    ('.', 'PUNCT'),
    ('uma', 'DET'),
    ('excelente', 'ADJ'),
    ('loja', 'NOUN'),
    ('de', 'ADP'),
    ('produtos', 'NOUN'),
    ('online', 'ADJ'),
    ('!', 'PUNCT')]


In [36]:
opx = load_oplexicon_data('oplexicon_v3.0/lexico_v3.0.txt')
print('Oplexicon size: ', len(opx))
print('Examples: ')

view = opx.items()
pp.pprint(list(view)[:7])

Oplexicon size:  15958
Examples: 
[   ('ab-rogar', [{'sentiment': '-1', 'tags': ['VERB'], 'word': ['ab-rogar']}]),
    ('ababadar', [{'sentiment': '0', 'tags': ['VERB'], 'word': ['ababadar']}]),
    (   'ababelar',
        [   {'sentiment': '-1', 'tags': ['VERB'], 'word': ['ababelar']},
            {'sentiment': '1', 'tags': ['VERB'], 'word': ['ababelar']}]),
    ('abacanar', [{'sentiment': '1', 'tags': ['VERB'], 'word': ['abacanar']}]),
    ('abacinar', [{'sentiment': '1', 'tags': ['VERB'], 'word': ['abacinar']}]),
    (   'abafar',
        [   {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['VERB'], 'word': ['abafar']}]),
    (   'abafante',
        [   {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafante']},
            {'s

In [0]:
ALEGRIA = ['abundante', 'acalmar', 'aceitável', 'aclamar', 'aconchego', 'adesão', 'admirar', 'adorar', 'afável', 'afeição', 'afeto', 'afortunado', 'agradar', 'ajeitar', 'alívio', 'amabilidade', 'amado', 'amar', 'amável', 'amenizar', 'ameno', 'amigável', 'amistoso', ' amizade', ' amor', ' animação', ' ânimo', 'anseio', 'ânsia', 'ansioso', 'apaixonado', 'apaziguar', 'aplausos', 'apoiar', 'aprazer', 'apreciar', 'aprovação', 'aproveitar', 'ardor', 'armirar', 'arrumar', 'atração', 'atraente', 'atrair', 'avidamente', 'avidez', 'ávido', 'belo', 'bem-estar', 'beneficência', 'beneficiador', 'benefício', 'benéfico', 'benevocência', 'benignamente', 'benígno', 'bom', 'bondade', 'bondoso', 'bonito', 'brilhante', 'brincadeira', 'calma', 'calor', 'caridade', 'caridoso', 'carinho', 'cativar', 'charme', 'cheery', 'clamar', 'cofortar', 'coleguismo', 'comédia', 'cômico', 'comover', 'compaixão', 'companheirismo', 'compatibilidade', 'compatível', 'complacência', 'completar', 'compreensão', 'conclusão', 'concretização', 'condescendência', 'confiança', 'confortante', 'congratulação', 'conquistar', 'consentir', 'consideração', 'consolação', 'contentamento', 'coragem', 'cordial', 'considerar', 'consolo', 'contente', 'cuidadoso', 'cumplicidade', 'dedicação', 'deleitado', 'delicadamente', 'delicadeza', 'delicado', 'desejar', 'despreocupação', 'devoção', 'devoto', 'diversão', 'divertido', 'encantar', 'elogiado', 'emoção', 'emocionante', 'emotivo', 'empatia', 'empático', 'empolgação', 'enamorar', 'encantado', 'encorajado', 'enfeitar', 'engraçado', 'entendimento', 'entusiasmadamente', 'entusiástico', 'esperança', 'esplendor', 'estima', 'estimar', 'estimulante', 'euforia', 'eufórico', 'euforizante', 'exaltar', 'excelente', 'excitar', 'expansivo', 'extasiar', 'exuberante', 'exultar', 'fã', 'facilitar', 'familiaridade', 'fascinação', 'fascínio', 'favor', 'favorecer', 'favorito', 'felicidade', 'feliz', 'festa', 'festejar', 'festivo', 'fidelidade', 'fiel', 'filantropia', 'filantrópico', 'fraterno', 'ganhar', 'generosidade', 'generoso', 'gentil', 'glória', 'glorificar', 'gostar', 'gostoso', 'gozar', 'gratificante', 'grato', 'hilariante', 'honra', 'humor', 'impressionar', 'incentivar', 'incentivo', 'inclinação', 'incrível', 'inspirar', 'interessar', 'interesse', 'irmandade', 'jovial', 'jubilante', 'júbilo', 'lealdade', 'legítimo', 'leveza', 'louvar', 'louvável', 'louvavelmente', 'lucrativo', 'lucro', 'maravilhoso', 'melhor', 'obter', 'obteve', 'ode', 'orgulho', 'paixão', 'parabenizar', 'paz', 'piedoso', 'positivo', 'prazenteiro', 'prazer', 'predileção', 'preencher', 'preferência', 'preferido', 'promissor', 'prosperidade', 'proteção', 'proteger', 'revigorar', 'simpático', 'vantajoso', 'protetor', 'risada', 'sobrevivência', 'vencedor', 'proveito', 'risonho', 'sobreviver', 'veneração', 'provilégio', 'romântico', 'sorte', 'ventura', 'querer', 'romantismo', 'sortudo', 'vida', 'radiante', 'saciar', 'sucesso', 'vigor', 'realizar', 'saciável', 'surpreender', 'virtude', 'recomendável', 'satisfação', 'tenro', 'virtuoso', 'reconhecer', 'satisfatoriamente', 'ternura', 'vitória', 'recompensa', 'satisfatório', 'torcer', 'vitorioso', 'recrear', 'satisfazer', 'tranquilo', 'viver', 'recreativo', 'satisfeito', 'tranquilo', 'vivo', 'recreação', 'sedução', 'triunfo', 'zelo', 'regozijar', 'seduzir', 'triunfal', 'zeloso', 'respeitar', 'sereno', 'triunfante', 'ressuscitar', 'simpaticamente', 'vantagem',]
DESGOSTO = ['abominável', 'adoentado', 'amargamente', 'antipatia', 'antipático', 'asco', 'asqueroso', 'aversão', 'chatear', 'chateação', 'desagrado', 'desagradável', 'desprezível', 'detestável', 'doente', 'doença', 'enfermidade', 'enjoativo', 'enjoo', 'enjôo', 'feio', 'fétido', 'golfar', 'grave', 'gravidade', 'grosseiro', 'grosso', 'horrível', 'ignóbil', 'ilegal', 'incomodar', 'incômdo', 'indecente', 'indisposição', 'indisposto', 'inescrupuloso', 'maldade', 'maldoso', 'malvado', 'mau', 'nauseabundo', 'nauseante', 'nausear', 'nauseoso', 'nojento', 'nojo', 'náusea', 'obsceno', 'obstruir', 'obstrução', 'ofensivo', 'patético', 'perigoso', 'repelente', 'repelir', 'repugnante', 'repulsa', 'repulsivo', 'repulsão', 'rude', 'sujeira', 'sujo', 'terrivelmente', 'terrível', 'torpe', 'travesso', 'travessura', 'ultrajante', 'vil', 'vomitar', 'vômito',]
MEDO = ['abominável', 'afugentar', 'alarmar', 'alerta', 'ameaça', 'amedrontar', 'angustia', 'angústia', 'angustiadamente', 'ansiedade', 'ansioso', 'apavorar', 'apreender', 'apreensão', 'apreensivo', 'arrepio', 'assombrado', 'assombro', 'assustado', 'assustadoramente', 'atemorizar', 'aterrorizante', 'brutal', 'calafrio', 'chocado', 'chocante', 'consternado', 'covarde', 'cruel', 'crueldade', 'cruelmente', 'cuidado', 'cuidadosamente', 'cuidadoso', 'defender', 'defensor', 'defesa', 'derrotar', 'desconfiado', 'desconfiança', 'desencorajar', 'desespero', 'deter', 'envergonhado', 'escandalizado', 'escuridão', 'espantoso', 'estremecedor', 'estremecer', 'expulsar', 'feio', 'friamente', 'fugir', 'hesitar', 'horrendo', 'horripilante', 'horrível', 'horrivelmente', 'horror', 'horrorizar', 'impaciência', 'impaciente', 'impiedade', 'impiedoso', 'indecisão', 'inquieto', 'insegurança', 'inseguro', 'intimidar', 'medonho', 'medroso', 'monstruosamente', 'mortalha', 'nervoso', 'pânico', 'pavor', 'premonição', 'preocupar', 'presságio', 'pressentimento', 'recear', 'receativamente', 'receio', 'receoso', 'ruim', 'suspeita', 'suspense', 'susto', 'temer', 'tenso', 'terror', 'tremor', 'temeroso', 'terrificar', 'timidamente', 'vigiar', 'temor', 'terrível', 'timidez', 'vigilante', 'tensão', 'terrivelmente', 'tímido',]
RAIVA = ['abominação', 'aborrecer', 'adredido', 'agredir', 'agressão', 'agressivo', 'amaldiçoado', 'amargor', 'amargura', 'amolar', 'angústia', 'animosidade', 'antipatia', 'antipático', 'asco', 'assassinar', 'assassinato', 'assediar', 'assédio', 'atormentar', 'avarento', 'avareza', 'aversão', 'beligerante', 'bravejar', 'chateação', 'chato', 'cobiçoso', 'cólera', 'colérico', 'complicar', 'contraiedade', 'contrariar', 'corrupção', 'corrupto', 'cruxificar', 'demoníaco', 'demônio', 'descaso', 'descontente', 'descontrole', 'desenganar', 'desgostar', 'desgraça', 'desprazer', 'desprezar', 'destruição', 'destruir', 'detestar', 'diabo', 'diabólico', 'doido', 'encolerizar', 'energicamente', 'enfurecido', 'enfuriante', 'enlouquecer', 'enraivecer', 'escandalizar', 'escândalo', 'escoriar', 'exasperar', 'execração', 'ferir', 'frustração', 'frustrar', 'fúria', 'furioso', 'furor', 'ganância', 'ganancioso', 'guerra', 'guerreador', 'guerrilha', 'hostil', 'humilhar', 'implicância', 'implicar', 'importunar', 'incomodar', 'incômodo', 'indignar', 'infernizar', 'inimigo', 'inimizade', 'injúria', 'injuriado', 'injustiça', 'insulto', 'malícia', 'odiável', 'repulsivo', 'inveja', 'malicioso', 'ódio', 'resmungar', 'ira', 'malignidade', 'odioso', 'ressentido', 'irado', 'malígno', 'ofendido', 'revolta', 'irascibilidade', 'maltratar', 'ofensa', 'ridículo', 'irascível', 'maluco', 'opressão', 'tempestuoso', 'irritar', 'malvadeza', 'opressivo', 'tirano', 'louco', 'malvado', 'oprimir', 'tormento', 'loucura', 'matar', 'perseguição', 'torturar', 'magoar', 'mesquinho', 'perseguir', 'ultrage', 'mal', 'misantropia', 'perturbar', 'ultrajar', 'maldade', 'misantrópico', 'perverso', 'vexatório', 'maldição', 'molestar', 'provocar', 'vigoroso', 'maldito', 'moléstia', 'rabugento', 'vingança', 'maldizer', 'mortal', 'raivoso', 'vingar', 'maldoso', 'morte', 'rancor', 'vingativo', 'maleficência', 'mortífero', 'reclamar', 'violência', 'maléfico', 'mortificar', 'repressão', 'violento', 'malevolência', 'nervoso', 'reprimir', 'zangar', 'malévolo', 'odiar', 'repulsa',]
SURPRESA = ['admirar', 'afeição', 'apavorante', 'assombro', 'chocado', 'chocante', 'desconcertar', 'deslumbrar', 'embasbacar', 'emudecer', 'encantamento', 'enorme', 'espanto', 'estupefante', 'estupefato', 'estupefazer', 'expectativa', 'fantasticamente', 'fantástico', 'horripilante', 'imaginário', 'imenso', 'impressionado', 'incrível', 'maravilha', 'milagre', 'mistério', 'misterioso', 'ótimo', 'pasmo', 'perplexo', 'prodígio', 'sensacional', 'surpreendente', 'surpreender', 'suspense', 'susto', 'temor', 'tremendo',]
TRISTEZA = ['abandonar', 'abatido', 'abominável', 'aborrecer', 'abortar', 'afligir', 'aflito', 'aflição', 'agoniar', 'amargo', 'amargor', 'amargura', 'ansiedade', 'arrepender', 'arrependidamente', 'atrito', 'azar', 'cabisbaixo', 'choro', 'choroso', 'chorão', 'coitado', 'compassivo', 'compunção', 'contristador', 'contrito', 'contrição', 'culpa', 'defeituoso', 'degradante', 'deplorável', 'deposição', 'depravado', 'depressivo', 'depressão', 'deprimente', 'deprimir', 'derrota', 'derrubar', 'desalentar', 'desamparo', 'desanimar', 'desapontar', 'desconsolo', 'descontente', 'desculpas', 'desencorajar', 'desespero', 'desgaste', 'desgosto', 'desgraça', 'desistir', 'desistência', 'deslocado', 'desmoralizar', 'desolar', 'desonra', 'despojado', 'desprazer', 'desprezo', 'desumano', 'desânimo', 'discriminar', 'disforia', 'disfórico', 'dissuadir', 'doloroso', 'dor', 'dó', 'enfadado', 'enlutar', 'entediado', 'entristecedor', 'entristecer', 'envergonhar', 'errante', 'erro', 'errôneo', 'escurecer', 'escuridão', 'escuro', 'esquecido', 'estragado', 'execrável', 'extirpar', 'falsidade', 'falso', 'falta', 'fraco', 'fraqueza', 'fricção', 'frieza', 'frio', 'funesto', 'fúnebre', 'grave', 'horror', 'humilhar', 'inconsolável', 'indefeso', 'infelicidade', 'infeliz', 'infortúnio', 'isolar', 'lacrimejante', 'lacrimoso', 'lamentar', 'lastimoso', 'luto', 'lutoso', 'lágrima', 'lástima', 'lúgubre', 'magoar', 'martirizar', 'martírio', 'mau', 'melancolia', 'melancólico', 'menosprezar', 'miseravelmente', 'misterioso', 'mistério', 'miséria', 'morre', 'morte', 'mortificante', 'mágoa', 'negligentemente', 'nocivo', 'obscuro', 'opressivo', 'opressão', 'oprimir', 'pena', 'penalizar', 'penitente', 'penoso', 'penumbra', 'perder', 'perturbado', 'perverso', 'pervertar', 'pesaroso', 'pessimamente', 'piedade', 'pobre', 'porcamente', 'prejudicado', 'prejudicial', 'prejuízo', 'pressionar', 'pressão', 'quebrar', 'queda', 'queixoso', 'rechaçar', 'remorso', 'repressivo', 'repressão', 'reprimir', 'ruim', 'secreto', 'servil', 'sobrecarga', 'sobrecarregado', 'sofrer', 'sofrimento', 'solidão', 'sombrio', 'soturno', 'sujo', 'suplicar', 'suplício', 'só', 'timidez', 'torturar', 'trevas', 'triste', 'tristemente', 'tédio', 'tímido', 'vazio',]

emotion_words = {
    'ALEGRIA': ALEGRIA,
    'DESGOSTO': DESGOSTO,
    'MEDO': MEDO,
    'RAIVA': RAIVA,
    'SURPRESA': SURPRESA,
    'TRISTEZA': TRISTEZA,
}
for key, values in emotion_words.items():
    for i, word in enumerate(values):
        emotion_words[key][i] = ''.join([remover_acentos(p.strip()) for p in LOOKUP.get(word.lower(), word.lower())])
# pp.pprint(emotion_words)

In [479]:
stpwords = stopwords.words('portuguese') + list(punctuation)
rms = ['um', 'não', 'mais', 'muito']
for rm in rms:
    del stpwords[stpwords.index(rm)]

def tokenize_frases(frase):
    return word_tokenize(remover_acentos(frase.lower()))

def rm_stop_words_tokenized(frase):
    frase = nlp(remover_acentos(frase.lower()))
    clean_frase = []
    for palavra in frase:
        if palavra.pos_ != 'PUNCT':
            palavra = palavra.lemma_
            for pc in ['.', ',']:
                palavra = palavra.replace(pc, '')
            if palavra not in stpwords and not palavra.isdigit():
                clean_frase.append(palavra)
    return ' '.join(clean_frase)

def generate_corpus(frases, tokenize=False):
    print('Iniciando processamento...')
    tokenized_frases = frases
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as procs:
        if tokenize:
            print('Executando processo de tokenização das frases...')
            tokenized_frases = procs.map(tokenize_frases, frases, chunksize=25)
        print('Executando processo de remoção das stopwords...')
        tokenized_frases = procs.map(rm_stop_words_tokenized, tokenized_frases, chunksize=25)
    print('Filtro e finalização...')
    return tokenized_frases


frases = [
    'Bom dia SENADOR, agora está claro porque o pedágio não baixava,o judiciário não se manifestava quando era provocado e as CPIs só serviram prá corrupção,deu no que deu 🙄',
    'Não basta apenas retirar o candidato preferencial da maioria dos eleitores brasileiros. Tem que impedir também que esses mesmos eleitores possam comparecer às urnas. Que democracia é essa, minha gente? Poder judiciário comprometido até os cabelos com o golpe de destrói o país.',
    'Deus abençoe o dia de todos você, tenham um bom trabalho e bom estudo a todos. E pra aqueles que não trabalha e nem estuda, boa curtição em sua cama 🙂',
    'Aprenda a ter amor próprio que nem essa banana q fez uma tatuagem dela mesma.',
    'Estou muito feliz hoje',
    'Dias chuvosos me deixam triste',
    'Hoje o dia esta excelente',
    'Tem certas coisas que eu não como, acho bem nojento ficar mastigando lingua de boi por exemplo',
    'É de se admirar àqueles que conseguem realizar boas ações sem desejar nada em troca',
]

# N = 10000

# frases += [' '.join(f).replace('_', ' ') for f in flt.sents()[:250]]
# frases += [' '.join(f).replace('_', ' ') for f in mch.sents()[:250]]
# frases += [' '.join(f).replace('_', ' ') for f in mcm.sents()[:250]]

frases = list(generate_corpus(frases, tokenize=False))
# print(frases)

ldocs = [f'D{i}' for i in range(len(frases))]

Iniciando processamento...
Executando processo de remoção das stopwords...
Filtro e finalização...


In [480]:
print('Tf-Idf:')
vectorizer = TfidfVectorizer(max_df=1, sublinear_tf=True, use_idf=True, ngram_range=(1, 1))
X_tfidf = vectorizer.fit_transform(frases)
print("   Actual number of tfidf features: %d" % X_tfidf.get_shape()[1])
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 3), index=vectorizer.get_feature_names(), columns=ldocs)
display(weights_tfidf.head(15))

Tf-Idf:
   Actual number of tfidf features: 77


,D0,D1,D2,D3,D4,D5,D6,D7,D8
abencoe,0.000,0.000,0.253,0.000,0.0,0.0,0.0,0.000,0.000
achar,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.302,0.000
acoes,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.378
admirar,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.378
agora,0.251,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000
amor,0.000,0.000,0.000,0.378,0.0,0.0,0.0,0.000,0.000
apenas,0.000,0.201,0.000,0.000,0.0,0.0,0.0,0.000,0.000
aprender,0.000,0.000,0.000,0.378,0.0,0.0,0.0,0.000,0.000
atar,0.000,0.201,0.000,0.000,0.0,0.0,0.0,0.000,0.000
baixar,0.251,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000


In [481]:
print('Count:')
vectorizer = CountVectorizer()
X_count = vectorizer.fit_transform(frases)
print("   Actual number of tfidf features: %d" % X_count.get_shape()[1])
weights_count = pd.DataFrame(X_count.toarray().T, index=vectorizer.get_feature_names(), columns=ldocs)
display(weights_count.head(15))

Count:
   Actual number of tfidf features: 84


,D0,D1,D2,D3,D4,D5,D6,D7,D8
abencoe,0,0,1,0,0,0,0,0,0
achar,0,0,0,0,0,0,0,1,0
acoes,0,0,0,0,0,0,0,0,1
admirar,0,0,0,0,0,0,0,0,1
agora,1,0,0,0,0,0,0,0,0
amor,0,0,0,1,0,0,0,0,0
apenas,0,1,0,0,0,0,0,0,0
aprender,0,0,0,1,0,0,0,0,0
atar,0,1,0,0,0,0,0,0,0
baixar,1,0,0,0,0,0,0,0,0


In [482]:
print('SVD: ')
AC = copy.deepcopy(X_count.toarray().T)
u, s, v = np.linalg.svd(AC, full_matrices=False)
print('Original and SVD equals: ', np.allclose(AC, np.dot(u, np.dot(np.diag(s), v))))

# print(AC)
# print(u.astype(np.float16))
# print('-' * 20)
# print(np.diag(s.astype(np.float16)))
# print('-' * 20)
# print(v.astype(np.float16))

SVD: 
Original and SVD equals:  True


In [483]:
print('Matriz U:')
weights_um = pd.DataFrame(u, index=vectorizer.get_feature_names(), columns=ldocs)
display(weights_um.head(15))

Matriz U:


,D0,D1,D2,D3,D4,D5,D6,D7,D8
abencoe,-0.121141,-0.078354,-0.107089,-0.015278,0.005196,0.044912,0.016150,-0.011787,-0.008395
achar,-0.020953,0.009936,0.010063,0.272938,-0.065397,0.009375,-0.008330,0.005523,0.003251
acoes,-0.014540,-0.011745,-0.010624,-0.015537,-0.137360,-0.330978,-0.022575,0.012702,0.006081
admirar,-0.014540,-0.011745,-0.010624,-0.015537,-0.137360,-0.330978,-0.022575,0.012702,0.006081
agora,-0.081228,-0.011599,0.188272,-0.026692,0.014662,-0.002089,0.021458,-0.015946,-0.011367
amor,-0.006683,0.004699,-0.010784,0.049210,0.323950,-0.139611,-0.001515,0.000986,0.000596
apenas,-0.062292,0.167098,-0.037964,-0.027951,-0.011140,0.001665,-0.003128,0.002280,0.001563
aprender,-0.006683,0.004699,-0.010784,0.049210,0.323950,-0.139611,-0.001515,0.000986,0.000596
atar,-0.062292,0.167098,-0.037964,-0.027951,-0.011140,0.001665,-0.003128,0.002280,0.001563
baixar,-0.081228,-0.011599,0.188272,-0.026692,0.014662,-0.002089,0.021458,-0.015946,-0.011367


In [484]:
print('Matriz VT:')
weights_vm = pd.DataFrame(v.T, columns=ldocs)
display(weights_vm.head(15))

Matriz VT:


,D0,D1,D2,D3,D4,D5,D6,D7,D8
0,-0.504540,-0.062465,0.852810,-0.095001,0.040895,-0.005758,0.047515,-0.031736,-0.016029
1,-0.386923,0.899867,-0.171965,-0.099485,-0.031072,0.004590,-0.006926,0.004539,0.002205
2,-0.752453,-0.421957,-0.485079,-0.054378,0.014493,0.123808,0.035762,-0.023458,-0.011839
3,-0.041513,0.025304,-0.048847,0.175148,0.903571,-0.384861,-0.003355,0.001963,0.000840
4,-0.001053,-0.000777,0.001357,-0.002037,0.004359,0.010365,-0.625093,-0.664798,-0.408862
5,-0.037402,-0.020154,0.023620,-0.019270,0.019013,0.043164,-0.532904,0.745000,-0.395007
6,-0.036408,-0.019436,0.022418,-0.017655,0.016478,0.037306,-0.564657,0.026027,0.822377
7,-0.130147,0.053509,0.045582,0.971442,-0.182406,0.025844,-0.018445,0.010991,0.004585
8,-0.090313,-0.063251,-0.048125,-0.055299,-0.383129,-0.912398,-0.049989,0.025280,0.008575


In [485]:
# hmm-lda
# https://ieeexplore.ieee.org/document/7363382
# https://link.springer.com/chapter/10.1007/978-3-642-21802-6_57
# http://www.ppgia.pucpr.br/~paraiso/Projects/Emocoes/Emocoes.html

emotion_weigths = {}
for k, items in enumerate(emotion_words.items()):
    key, values = items
    emotion_weigths[key] = []
    for i, word in enumerate(values):
        idx_val = 0
        w = 0
        try:
            index = weights_count.index.get_loc(word)
            idx_val = u[index]
            w = weights_count.iloc[index].values
        except:
            w = np.zeros((len(ldocs, )))
        emotion_weigths[key].append(np.mean(idx_val * w))
# pp.pprint(emotion_weigths)

values = np.zeros((277, 6))
for k, val in enumerate(emotion_weigths.values()):
    for t, va in enumerate(val):
        values[t, k] = va
df = pd.DataFrame(values, columns=emotion_weigths.keys())
display(df[
    (df['ALEGRIA'] != 0) | (df['DESGOSTO'] != 0) | (df['MEDO'] != 0) | 
    (df['RAIVA'] != 0) | (df['SURPRESA'] != 0) | (df['TRISTEZA'] != 0)
])

,ALEGRIA,DESGOSTO,MEDO,RAIVA,SURPRESA,TRISTEZA
0,0.000000,0.000000,0.0,0.000000,0.000676,0.000000
6,0.000676,0.000000,0.0,0.000000,0.000000,0.000000
24,0.005468,0.000000,0.0,0.000000,0.000000,0.000000
33,0.000000,0.000000,0.0,-0.009025,0.000000,0.000000
44,0.000000,0.000614,0.0,0.000000,0.000000,0.000000
56,-0.102280,0.000000,0.0,0.000000,0.000000,0.000000
106,0.000676,0.000000,0.0,0.000000,0.000000,0.000000
137,-0.028333,0.000000,0.0,0.000000,0.000000,0.000000
138,0.000000,0.000000,0.0,-0.009025,0.000000,0.000000
152,0.000174,0.000000,0.0,0.000000,0.000000,0.000000


In [486]:
print(v.T[5,])
dtframe = {d: np.zeros(len(ldocs)) for d in emotion_weigths}
for k, item in enumerate(emotion_weigths.items()):
    sent = np.array(item[1])
    i = 0
    for j in range(len(ldocs)):
        similarity = cosine_similarity(v.T, [sent])
        for m in similarity:
            i += 1
            dtframe[item[0]][i-1] = m

for frase in frases:
    print(f'-{frase}')

df = pd.DataFrame(list(dtframe.values()), index=dtframe.keys(), columns=ldocs)
display(df.head(15))

[-0.03740167 -0.02015354  0.02361973 -0.01927029  0.01901313  0.04316394
 -0.53290366  0.74499971 -0.3950066 ]


ValueError: ignored

In [0]:
print("LSA using TruncatedSVD:")

# Project the tfidf vectors onto the first N principal components.
# Though this is significantly fewer features than the original tfidf vector,
# they are stronger features, and the accuracy is higher.
svd = TruncatedSVD(50)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_lsa = lsa.fit_transform(X_count)

explained_variance = svd.explained_variance_ratio_.sum()
print("   Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

print(svd.explained_variance_.shape)
print(svd.singular_values_.shape) # S
print(svd.components_.shape) # VT

LSA using TruncatedSVD:
   Explained variance of the SVD step: 100%
(4,)
(4,)
(4, 61)


In [0]:
print('LSA using numpy:')
u, s, v = np.linalg.svd(X_tfidf.toarray(), full_matrices=False)
print(u)
print(s.shape)
print(v.shape)

LSA using numpy:
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
(4,)
(4, 58)


In [0]:
print('LSA using scikit-learn randomized_svd:')
U, Sigma, VT = randomized_svd(X_count, 
                              n_components=50,
                              n_iter=5,
                              random_state=None)
print(U.shape)
print(Sigma.shape)
print(VT.shape)

print(U)
# print(VT)

LSA using scikit-learn randomized_svd:
(4, 4)
(4,)
(4, 61)
[[ 8.23887410e-02  4.64153487e-01  8.81914756e-01  0.00000000e+00]
 [ 9.96228666e-01 -6.25206048e-02 -6.01632624e-02 -3.70560802e-16]
 [ 2.72128558e-02  8.83545536e-01 -4.67554003e-01  2.07365235e-16]
 [ 3.63520293e-16 -2.06384313e-16  7.46602991e-17  1.00000000e+00]]


In [0]:
# define a matrix
# A = array([[1, 2], [3, 4], [5, 6]])
A = np.array([
    [1, 1, 1, 0, 0],
    [3, 3, 3, 0, 0],
    [4, 4, 4, 0, 0],
    [5, 5, 5, 0, 0],
    [0, 2, 0, 4, 4],
    [0, 0, 0, 5, 5],
    [0, 1, 0, 2, 2],
])
print(A.shape)


u, s, v = np.linalg.svd(copy.deepcopy(A), full_matrices=False)

print(u.astype(np.float16))
print('-' * 20)
print(np.diag(s.astype(np.float16)))
print('-' * 20)
print(v.astype(np.float16))

(7, 5)
[[-0.1376  -0.0236  -0.01081  0.56    -0.3757 ]
 [-0.4128  -0.07086 -0.03244  0.2064   0.756  ]
 [-0.5503  -0.0944  -0.04324 -0.7246  -0.1846 ]
 [-0.688   -0.11804 -0.05405  0.344   -0.2307 ]
 [-0.1528   0.5913   0.654    0.       0.2    ]
 [-0.0722   0.7314  -0.678    0.       0.     ]
 [-0.0764   0.2957   0.327    0.      -0.4    ]]
--------------------
[[12.484  0.     0.     0.     0.   ]
 [ 0.     9.51   0.     0.     0.   ]
 [ 0.     0.     1.346  0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]]
--------------------
[[-0.5625  -0.593   -0.5625  -0.09015 -0.09015]
 [-0.1266   0.02878 -0.1266   0.6953   0.6953 ]
 [-0.4097   0.8047  -0.4097  -0.09125 -0.09125]
 [-0.707    0.       0.707    0.       0.     ]
 [ 0.      -0.       0.      -0.707    0.707  ]]


In [0]:
u, s, v = randomized_svd(copy.deepcopy(A), 
                          power_iteration_normalizer='auto',
                          flip_sign=True,
                          n_components=100,
                          n_iter=1,
                          random_state=None)
print(u.astype(np.float16))
print('-' * 20)
print(np.diag(s.astype(np.float16)))
print('-' * 20)
print(v.astype(np.float16))

[[ 1.3757e-01 -2.3605e-02  1.0811e-02  9.3652e-01  2.8711e-01]
 [ 4.1284e-01 -7.0862e-02  3.2440e-02 -3.1152e-01  8.4912e-01]
 [ 5.5029e-01 -9.4421e-02  4.3243e-02  1.2622e-01 -2.9834e-01]
 [ 6.8799e-01 -1.1804e-01  5.4047e-02 -1.0132e-01 -3.2812e-01]
 [ 1.5283e-01  5.9131e-01 -6.5381e-01 -1.5199e-04 -6.2406e-05]
 [ 7.2205e-02  7.3145e-01  6.7822e-01  0.0000e+00  0.0000e+00]
 [ 7.6416e-02  2.9565e-01 -3.2690e-01  3.0398e-04  1.2481e-04]]
--------------------
[[12.484  0.     0.     0.     0.   ]
 [ 0.     9.51   0.     0.     0.   ]
 [ 0.     0.     1.346  0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]]
--------------------
[[ 0.5625   0.593    0.5625   0.09015  0.09015]
 [-0.1266   0.02878 -0.1266   0.6953   0.6953 ]
 [ 0.4097  -0.8047   0.4097   0.09125  0.09125]
 [ 0.707    0.      -0.707   -0.      -0.     ]
 [-0.      -0.       0.       0.707   -0.707  ]]


In [0]:
u, s, v = svd(copy.deepcopy(A))
print(u.astype(np.float16))
print('-' * 20)
print(np.diag(s.astype(np.float16)))
print('-' * 20)
print(v.astype(np.float16))

[[-0.1376  -0.0236  -0.01081  0.56    -0.3757  -0.7     -0.1879 ]
 [-0.4128  -0.07086 -0.03244  0.2064   0.756   -0.258    0.378  ]
 [-0.5503  -0.0944  -0.04324 -0.7246  -0.1846  -0.344   -0.0923 ]
 [-0.688   -0.11804 -0.05405  0.344   -0.2307   0.57    -0.11536]
 [-0.1528   0.5913   0.654    0.       0.2      0.      -0.4    ]
 [-0.0722   0.7314  -0.678    0.       0.       0.       0.     ]
 [-0.0764   0.2957   0.327    0.      -0.4      0.       0.8    ]]
--------------------
[[12.484  0.     0.     0.     0.   ]
 [ 0.     9.51   0.     0.     0.   ]
 [ 0.     0.     1.346  0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]]
--------------------
[[-0.5625  -0.593   -0.5625  -0.09015 -0.09015]
 [-0.1266   0.02878 -0.1266   0.6953   0.6953 ]
 [-0.4097   0.8047  -0.4097  -0.09125 -0.09125]
 [-0.707    0.       0.707    0.       0.     ]
 [ 0.      -0.       0.      -0.707    0.707  ]]


In [0]:
# SVD
U, s, VT = svd(A)
print(U.astype(np.float16))
print(np.diag(s.astype(np.float16)))
print(VT.astype(np.float16))

[[-0.1376  -0.0236  -0.01081  0.56    -0.3757  -0.7     -0.1879 ]
 [-0.4128  -0.07086 -0.03244  0.2064   0.756   -0.258    0.378  ]
 [-0.5503  -0.0944  -0.04324 -0.7246  -0.1846  -0.344   -0.0923 ]
 [-0.688   -0.11804 -0.05405  0.344   -0.2307   0.57    -0.11536]
 [-0.1528   0.5913   0.654    0.       0.2      0.      -0.4    ]
 [-0.0722   0.7314  -0.678    0.       0.       0.       0.     ]
 [-0.0764   0.2957   0.327    0.      -0.4      0.       0.8    ]]
[[12.484  0.     0.     0.     0.   ]
 [ 0.     9.51   0.     0.     0.   ]
 [ 0.     0.     1.346  0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[-0.5625  -0.593   -0.5625  -0.09015 -0.09015]
 [-0.1266   0.02878 -0.1266   0.6953   0.6953 ]
 [-0.4097   0.8047  -0.4097  -0.09125 -0.09125]
 [-0.707    0.       0.707    0.       0.     ]
 [ 0.      -0.       0.      -0.707    0.707  ]]


In [0]:
A = [
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8]
]

U, s, VT = svd(A)
print(U.astype(np.float16))
print(np.diag(s.astype(np.float16)))
print(VT.T.astype(np.float16))

[[-0.1525  -0.8228  -0.3945  -0.38   ]
 [-0.3499  -0.4214   0.2428   0.801  ]
 [-0.5474  -0.0201   0.6978  -0.4614 ]
 [-0.7446   0.381   -0.5464   0.04074]]
[[14.266  0.   ]
 [ 0.     0.627]]
[[-0.6416  0.767 ]
 [-0.767  -0.6416]]


In [0]:
A = [
    [2, 4],
    [1, 3],
    [0, 0],
    [0, 0]
    # [0, -1],
    # [-2, 1],
    # [1, 0]
]

U, s, VT = svd(A, full_matrices=False)
print(U.astype(np.float16))
print(np.diag(s.astype(np.float16)))
print(VT.T.astype(np.float16))

print(np.dot(U, U.T).astype(np.float16))
print(np.dot(VT, VT.T).astype(np.float16))


print(np.allclose(A, np.dot(U, np.dot(np.diag(s), VT))))

print(np.dot(U, np.dot(np.diag(s), VT)).astype(np.float16))

[[-0.8174 -0.576 ]
 [-0.576   0.8174]
 [ 0.      0.    ]
 [ 0.      0.    ]]
[[5.465 0.   ]
 [0.    0.366]]
[[-0.4045 -0.9146]
 [-0.9146  0.4045]]
[[ 1. -0.  0.  0.]
 [-0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[ 1. -0.]
 [-0.  1.]]
True
[[2. 4.]
 [1. 3.]
 [0. 0.]
 [0. 0.]]
